In [1]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [3]:
#from tensorflow.keras import models
#from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [4]:
df_input = pd.read_csv("CD3D_100-150℃_0.2-1.6mm_0.4-0.9V_std", usecols=[1,2,3,4,5,6])

In [5]:
df_output = pd. read_csv("CD3D_100-150℃_0.2-1.6mm_0.4-0.9V", usecols=[7])

In [6]:
df_input.shape

(5940000, 6)

In [7]:
df_output.shape

(5940000, 1)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(df_input, df_output, test_size=0.1, random_state=0)

In [9]:
print(x_train.shape,x_test.shape)

(5346000, 6) (594000, 6)


In [10]:
x_train

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm]
5398873,-0.899747,-1.615496,-0.386727,-9.486833e-01,1.46385,-0.267261
5593541,0.598857,0.209366,-0.088889,1.581139e+00,1.46385,-0.267261
1157183,-1.062096,1.266899,1.298443,-3.162278e-01,-0.87831,-1.069045
4758896,1.214950,0.696630,-1.418201,2.808667e-15,0.87831,1.336306
5713770,1.631228,0.131113,-0.902464,-9.486833e-01,1.46385,1.336306
...,...,...,...,...,...,...
2249467,-1.145352,-1.211555,1.130829,1.264911e+00,-0.29277,-1.069045
5157699,-0.046376,-1.681710,1.538261,6.324555e-01,1.46385,-1.069045
2215104,-0.895585,-0.443909,1.418352,3.162278e-01,-0.29277,-1.069045
1484405,-0.025562,0.696630,-0.411225,-3.162278e-01,-0.87831,-0.267261


In [11]:
y_train

,i[A/m^2]
5398873,20750.577957
5593541,2065.708599
1157183,10044.370147
4758896,5115.924705
5713770,8771.056553
...,...
2249467,1348.419943
5157699,7995.948169
2215104,6462.545799
1484405,7909.542765


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers, initializers

model = Sequential()

weights = [com.get_weights() for com in model.layers[0:]]  #重さを抽出（※始めに回すときだけ下の行と入れ替える）



model.add(Dense(units=32, input_shape=(6,), activation='relu'))  
model.add(Dense(units=10, activation='relu'))  #
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1, activation='linear'))  #隠れ層3層でやってみる

model.compile(loss='mean_absolute_error',    
              optimizer='adam',             #とりあえずadamで
              metrics=['mae'],
              run_eagerly=False)   #計算時間5倍くらいかかる

In [13]:
callbacks1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_mae',
                            factor=0.2,
                            patience=5,
                            mode="auto",
                            min_lr=0.001)
callbacks2 = tf.keras.callbacks.EarlyStopping(monitor="val_mae",
                                              patience=20,
                                              mode="auto")
history =  model.fit(x=x_train,
                     y=y_train,
                     epochs = 20000,
                     batch_size=200,
                     validation_data=(x_test, y_test),
                     callbacks=[callbacks1,callbacks2])

Epoch 1/20000
26730/26730 [==============================] - 35s 1ms/step - loss: 787.7426 - mae: 787.7426 - val_loss: 306.7122 - val_mae: 306.7122 - lr: 0.0010
Epoch 2/20000
26730/26730 [==============================] - 37s 1ms/step - loss: 199.3098 - mae: 199.3098 - val_loss: 143.5145 - val_mae: 143.5145 - lr: 0.0010
Epoch 3/20000
26730/26730 [==============================] - 38s 1ms/step - loss: 134.7582 - mae: 134.7582 - val_loss: 129.1546 - val_mae: 129.1546 - lr: 0.0010
Epoch 4/20000
26730/26730 [==============================] - 46s 2ms/step - loss: 125.6377 - mae: 125.6377 - val_loss: 122.4915 - val_mae: 122.4915 - lr: 0.0010
Epoch 5/20000
26730/26730 [==============================] - 38s 1ms/step - loss: 120.8454 - mae: 120.8454 - val_loss: 118.8817 - val_mae: 118.8817 - lr: 0.0010
Epoch 6/20000
26730/26730 [==============================] - 38s 1ms/step - loss: 118.3426 - mae: 118.3426 - val_loss: 118.6469 - val_mae: 118.6469 - lr: 0.0010
Epoch 7/20000
26730/26730 [=======

In [14]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'mae', 'val_loss', 'val_mae', 'lr'])

In [15]:
weights = [com.get_weights() for com in model.layers[0:]] 
model.layers[0].set_weights(weights[0])
model.layers[1].set_weights(weights[1])
model.layers[2].set_weights(weights[2])

In [16]:
model.save('i_predict CD_3Ddist2')

INFO:tensorflow:Assets written to: i_predict CD_3Ddist2\assets
